<a href="https://colab.research.google.com/github/mohammed21kamall/Graduation-Project/blob/main/PartOfPredict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
# Define the prediction function using the provided equation
def predict_rating(ru, rv, rvi, Suv, degree_v, n_nearest_users, cu, cv):

    sum_weights = 0
    sum_rating_diff_weighted = 0

    for v in range(1, n_nearest_users + 1):
        sum_rating_diff_weighted += (rvi - rv) * Suv * w(cu, cv)  # Numerator
        sum_weights += (degree_v + 1) * abs(Suv)  # Denominator

    if sum_weights == 0:
        return 0  # Avoid division by zero

    predicted_rating = ru + (sum_rating_diff_weighted / sum_weights)
    return predicted_rating

The Similarity (Suv)

In [3]:
# Similarity
model_filename = 'drive/MyDrive/Dataset2/SimilarityOfUsers.csv'
Suv = pd.read_csv(model_filename)

Suv = Suv.clip(upper=1.0)
Suv = Suv.applymap(lambda x: max(0, x))
Suv = Suv.applymap(lambda x: x if x >= 0.4 else float(0.0))

Suv.head()

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,1.000000,0.0,0.0,0.516196,0.0,0.0,0.540215,0.0,0.0,0.0,...,0.0,0.480139,0.651735,0.000000,0.436931,0.00000,1.0,1.00000,0.000000,0.410598
1,0.000000,1.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
2,0.000000,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.000000,0.000000
3,0.516196,0.0,0.0,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,1.000000,0.000000,0.000000,0.88533,0.0,0.47253,0.000000,0.000000
4,0.000000,0.0,0.0,0.000000,1.0,1.0,0.000000,1.0,0.0,0.0,...,0.0,1.000000,0.000000,0.724984,0.000000,0.00000,0.0,0.00000,0.526469,0.000000


In [24]:
threshold=0.8
# Filter similar user IDs for a specified user ID
def get_similar_user_ids(user_id, threshold):
    # Filter the DataFrame to get similarity values for the specified user ID
    similarities_to_user = Suv[user_id]

    # Filter similar user IDs with similarity >= threshold
    similar_user_ids = similarities_to_user[similarities_to_user >= threshold].index.tolist()

    # Remove the specified user ID from the list if it's present
    similar_user_ids.remove(user_id) if user_id in similar_user_ids else None

    # Increase each user ID by 1
    similar_user_ids = [(int(user) + 1) for user in similar_user_ids]

    return similar_user_ids

# Prompt the user to enter the user ID
user_id = input("Enter the user ID: ")

# Check if the entered user ID exists in the DataFrame
if user_id not in Suv.columns:
    print("User ID not found!")
else:
    # Get similar user IDs for the specified user ID
    similar_user_ids = list(get_similar_user_ids(user_id, threshold))

    # Display similar user IDs
    print("Similar User IDs:", similar_user_ids)


Enter the user ID: 5
Similar User IDs: [5, 6, 8, 33, 38, 40, 43, 46, 54, 58, 84, 94, 102, 107, 109, 117, 121, 126, 130, 133, 134, 142, 144, 145, 162, 170, 173, 174, 179, 191, 229, 235, 240, 242, 262, 273, 314, 323, 347, 349, 353, 372, 373, 379, 385, 394, 404, 411, 436, 444, 446, 447, 455, 458, 468, 470, 476, 498, 507, 512, 541, 559, 565, 566, 574, 584, 588, 592, 602]


In [25]:
# Function to get all movie IDs rated by users in a list
def get_list_rated_movie_ids(user_ids):
    rated_movies_by_users = {}  # Dictionary to store rated movies by each user

    for user_id in user_ids:
        # Filter ratings DataFrame for the specified user ID
        user_ratings = ratings[ratings['userId'] == user_id]

        # Extract and store the unique movie IDs rated by the user
        rated_movie_ids = user_ratings['movieId'].unique()
        rated_movies_by_users[user_id] = rated_movie_ids

    return rated_movies_by_users

# Example: Provide a list of user IDs
list_id = similar_user_ids  # Example list of user IDs

# Get all movie IDs rated by the specified users
rated_movies_by_users = get_list_rated_movie_ids(list_id)

# Print the movie IDs rated by each user
for user_id, rated_movies in rated_movies_by_users.items():
    print(f"Movie IDs rated by User {user_id}:")
    print(rated_movies)
    print("\n")

Movie IDs rated by User 5:
[  1  21  34  36  39  50  58 110 150 153 232 247 253 261 265 266 290 296
 300 316 318 344 349 357 364 367 380 410 457 474 475 515 527 531 534 588
 589 590 592 594 595 596 597 608]


Movie IDs rated by User 6:
[   2    3    4    5    6    7    8   10   11   13   15   16   17   19
   21   22   24   25   26   27   31   32   34   36   41   43   45   46
   47   50   54   60   61   62   65   66   76   79   86   87   88   89
   92   93   95  100  102  104  105  110  112  113  126  135  140  141
  145  146  150  151  153  158  159  160  161  163  165  168  170  171
  174  177  179  180  181  185  186  189  191  195  196  201  204  205
  207  208  209  210  212  216  217  218  219  222  224  225  230  231
  234  236  237  239  240  243  248  250  251  252  253  254  256  257
  258  261  262  265  266  267  270  271  273  274  276  277  279  281
  282  288  289  291  292  293  296  302  303  304  310  312  313  314
  315  316  317  318  327  329  330  332  333  336  33

In [26]:
def get_users_rated_movie(movie_id, rated_movies_by_users):
    """
    Get the list of user IDs who have rated the given movie.

    Args:
    - movie_id: The movie ID to search for.
    - rated_movies_by_users: Dictionary containing movie IDs rated by each user.

    Returns:
    - users_rated_movie: List of user IDs who have rated the given movie.
    """
    users_rated_movie = []
    for user_id, rated_movies in rated_movies_by_users.items():
        if movie_id in rated_movies:
            users_rated_movie.append(user_id)
    return users_rated_movie

# Example: Provide a movie ID
movie_id = 1

# Get the list of user IDs who have rated the given movie
users_rated_movie = get_users_rated_movie(movie_id, rated_movies_by_users)

# Print the list of user IDs who have rated the given movie
print(f"Users who have rated movie ID {movie_id}:")
print(users_rated_movie)


Users who have rated movie ID 1:
[5, 33, 40, 43, 46, 54, 107, 121, 130, 134, 144, 145, 179, 191, 229, 240, 273, 314, 323, 347, 353, 372, 373, 385, 411, 436, 468, 470, 476, 541, 559, 584]


In [28]:

# 0.8 is the threshold i.e the cosine similarity should be at least greater than 0.8 between two users.
n = int(input("Enter the number of similar user IDs to display: "))
print("\nSimilarity Values: \n")
print("User ID" +" "+ "Similarity")
similarity_values = Suv[Suv[user_id] >= 0.8][user_id].sort_values(ascending=False)[0:n]
print(similarity_values)


Enter the number of similar user IDs to display: 5

Similarity Values: 



KeyError: 602

In [29]:
similarity_values

4      1.0
372    1.0
428    1.0
Name: 6, dtype: float64

In [6]:
print(f"The Values For All Greater Users For User ID {user_id}")
similarity_values_list = similarity_values.tolist()
print(similarity_values_list)

The Values For All Greater Users For User ID 6
[1.0, 1.0, 1.0]


In [7]:
# Extract user IDs from the index of similarity_values and append them to a list
print(f"All Greater User IDs For User ID {user_id} :")
user_ids_list = []
for index in similarity_values.index:
    user_ids_list.append(index)
print("User IDs:", user_ids_list)

All Greater User IDs For User ID 6 :
User IDs: [4, 372, 428]


Rating U (ru) & Rating V (rv)

In [8]:
ratings = pd.read_csv("drive/MyDrive/Dataset2/ratings.csv")
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [10]:
# Function to get rating for a user and movie
def get_rating(ratings, user_id, movie_id):
    rating = ratings[(ratings['userId'] == user_id) & (ratings['movieId'] == movie_id)]['rating'].values
    if len(rating) > 0:
        return rating[0]
    else:
        return None

# Prompt the user for input for two users and a movie
movie_id = int(input("Enter the movieId: "))
ratings_list = []

for i in range(n):
    user_id = user_ids_list[i]
    rating = get_rating(ratings, user_id, movie_id)
    ratings_list.append(rating)

# Print all ratings
print("Ratings for movie", movie_id, "are:", ratings_list)

Enter the movieId: 260
Ratings for movie 260 are: [5.0, 4.0, 3.5]


All Movies ID For User


In [13]:
movie = pd.read_csv("drive/MyDrive/Dataset2/movies.csv")
movie = movie[['movieId', 'title']]
movie.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


Mean Of Ratings (ru & rv)

In [15]:
mean_ratings_by_user = ratings.groupby('userId')['rating'].mean().reset_index()
mean_ratings_by_user.head()

,userId,rating
0,1,4.366379
1,2,3.948276
2,3,2.435897
3,4,3.555556
4,5,3.636364
